In [100]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import LinearConstraint, Bounds, minimize

try:
    import pdblp
    con = pdblp.BCon(debug=True, port=8194, timeout=5000)
    con.start()
    con.debug = False
except:
    print("connection failed")

pdblp.pdblp:INFO:Event Type: 'SESSION_STATUS'
pdblp.pdblp:INFO:Message Received:
SessionConnectionUp = {
    server = "localhost:8194"
    serverId = "bbcomm-psd-finmath-15-606707"
    encryptionStatus = "Clear"
    compressionStatus = "Uncompressed"
}

pdblp.pdblp:INFO:Event Type: 'SESSION_STATUS'
pdblp.pdblp:INFO:Message Received:
SessionStarted = {
    initialEndpoints[] = {
        initialEndpoints = {
            address = "localhost:8194"
        }
    }
}

pdblp.pdblp:INFO:Event Type: 'SERVICE_STATUS'
pdblp.pdblp:INFO:Message Received:
CID: {[ valueType=AUTOGEN classId=0 value=6 ]}
RequestId: d0cc6d23-64a0-c1ec-0000-1224aac00600
ServiceOpened = {
    serviceName = "//blp/refdata"
}

pdblp.pdblp:INFO:Event Type: 'SERVICE_STATUS'
pdblp.pdblp:INFO:Message Received:
CID: {[ valueType=AUTOGEN classId=0 value=8 ]}
RequestId: 97ccb423-64a1-c1ec-0000-1224aac00600
ServiceOpened = {
    serviceName = "//blp/exrsvc"
}



In [114]:
start_date = '20240706'
end_date = '20241007'
# rf = 3.78/100

def valid_date(idx):
    return len(str(idx).split('-')) ==3

stocks = {
    'Tech': ['NVDA', 'DELL', 'INTC', 'QCOM', 'DBX'],
    'Middle_East_conflict': ['CVX', 'TRGP', 'LMT'],
    'Health_care': ['BSX', 'LLY', 'AMGN', 'BMY'],
    'Financial_services': ['TIGR', 'BRK/B'],
    'Consumer': ['JKS', 'LULU']
}

field = ['TOT_RETURN_INDEX_NET_DVDS'] 

returns = pd.DataFrame()

try:
    returns = pd.read_csv(r'stock_returns.csv', header=0, index_col=0)
    returns = returns[returns.index.map(valid_date)]
    returns.index = pd.to_datetime(returns.index)
    
except FileNotFoundError:

    for category, tickers in stocks.items():
        print(f"Category: {category}")
        for ticker in tickers:
            bloomberg_ticker = f"{ticker} US Equity"
            print(bloomberg_ticker)
            data = con.bdh(bloomberg_ticker, field, start_date, end_date)
            print(f"Data for {bloomberg_ticker}:")
            print(data.head)
            # data = data[ticker][data[ticker].columns[0]]
            if returns.empty:
                returns=pd.DataFrame(data)
            else:
                returns = returns.merge(data, left_index=True, right_index=True, how = 'left')
    returns = returns[returns.index.map(valid_date)]
    returns.to_csv(r'stock_returns.csv')

returns.head

<bound method NDFrame.head of            NVDA US Equity DELL US Equity INTC US Equity QCOM US Equity  \
ticker                                                                   
2024-07-08          128.2         145.97          33.99         207.89   
2024-07-09         131.38         145.74          34.59         207.12   
2024-07-10         134.91         145.77          34.87          208.8   
2024-07-11          127.4         140.72           33.5         199.85   
2024-07-12         129.24         139.57          34.49         202.43   
...                   ...            ...            ...            ...   
2024-10-01       117.0098       113.5601        22.8394       166.6408   
2024-10-02         118.86       113.8612        22.5374        169.194   
2024-10-03       122.8603        116.079        22.4065       169.7971   
2024-10-04       124.9305       120.8458        22.7387        169.777   
2024-10-07       127.7307       119.5111        22.5273       167.8068   

       

In [116]:
returns


,NVDA US Equity,DELL US Equity,INTC US Equity,QCOM US Equity,DBX US Equity,CVX US Equity,TRGP US Equity,LMT US Equity,BSX US Equity,LLY US Equity,AMGN US Equity,BMY US Equity,TIGR US Equity,BRK/B US Equity,JKS US Equity,LULU US Equity
ticker,,,,,,,,,,,,,,,,
2024-07-08,128.2,145.97,33.99,207.89,22.24,154.33,132.55,461.76,76.11,918.0,311.46,39.99,4.33,409.08,21.12,294.03
2024-07-09,131.38,145.74,34.59,207.12,22.02,152.98,132.41,460.3,76.51,932.5,315.91,40.86,4.39,410.52,21.67,289.87
2024-07-10,134.91,145.77,34.87,208.8,21.87,155.13,132.89,461.01,77.11,939.78,321.67,41.12,4.67,413.79,22.04,288.08
2024-07-11,127.4,140.72,33.5,199.85,22.32,155.26,133.61,460.38,76.89,934.14,325.09,40.75,4.61,418.78,22.9,289.2
2024-07-12,129.24,139.57,34.49,202.43,22.68,155.59,135.05,463.73,77.69,948.4,330.83,40.46,4.53,424.44,23.21,291.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-01,117.0098,113.5601,22.8394,166.6408,25.05,151.3619,151.4148,609.225,84.14,885.7143,322.562,52.9,6.23,457.41,27.4938,266.45
2024-10-02,118.86,113.8612,22.5374,169.194,25.12,152.6257,154.0896,606.55,84.49,892.3135,321.968,53.76,8.06,456.67,27.4938,260.53
2024-10-03,122.8603,116.079,22.4065,169.7971,25.53,152.9291,156.3621,609.184,83.82,886.7858,319.702,54.2,9.19,452.96,28.0969,262.56


In [120]:
# Calculate daily returns
returns = returns.pct_change().dropna()

# Calculate expected returns (annualized)
expected_returns = returns.mean() * 252

# Calculate the covariance matrix (annualized)
cov_matrix = returns.cov() * 252

# Function to calculate portfolio performance
def portfolio_performance(weights, expected_returns, cov_matrix):
    portfolio_return = np.dot(weights, expected_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return portfolio_return, portfolio_volatility

# Function to minimize (negative Sharpe ratio)
def minimize_volatility(weights, expected_returns, cov_matrix):
    return portfolio_performance(weights, expected_returns, cov_matrix)[1]

# Constraints: weights must sum to 1
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
# Bounds: weights must be between 0 and 0.2
bounds = tuple((0, 0.2) for _ in range(len(stock_list)))

# Initial guess: equally distributed weights
init_guess = len(stock_list) * [1. / len(stock_list)]

# Optimization to minimize volatility (mean-variance optimization)
optimized_result = minimize(minimize_volatility, init_guess, args=(expected_returns, cov_matrix),
                            method='SLSQP', bounds=bounds, constraints=constraints)

# Get the optimal weights
optimal_weights = optimized_result.x

# Calculate expected return and volatility for the optimized portfolio
opt_return, opt_volatility = portfolio_performance(optimal_weights, expected_returns, cov_matrix)

# Display results
res = dict(zip(stock_list, optimal_weights))
print("Optimal Weights:", res)
print("Expected Portfolio Return:", opt_return)
print("Expected Portfolio Volatility:", opt_volatility)

# Plot the optimal portfolio weights

plt.figure(figsize=(10,6))
plt.bar(stock_list, optimal_weights, color='skyblue')
plt.title('Optimal Portfolio Weights')
plt.xlabel('Stocks')
plt.ylabel('Weight')
plt.xticks(rotation=45)
plt.show()


'''
yvals = []
for i in range(len(test_data.index)):
    portfolio_returns = -1
    for stock in stock_list:
        portfolio_returns += res[stock] * test_data[stock].iloc[i] / data[stock].iloc[-1]
    yvals.append(portfolio_returns)
xvals = np.linspace(0, len(yvals)-1, len(yvals))
plt.bar(xvals, yvals)
'''

#  above is for testing on test data

TypeError: unsupported operand type(s) for /: 'str' and 'str'